### Install sagemaker-experiments

In [1]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

In [2]:
import sagemaker
import boto3

session = sagemaker.Session()
sm = boto3.Session().client('sagemaker')
role = sagemaker.get_execution_role()

### Split Data

In [3]:
import pandas as pd

RANDOM_STATE = 99

DATA_FILE = './aws_sagemaker/scikit-learn/classification/iris_parameterized/data/iris.csv'

# load csv in memory
data = pd.read_csv(DATA_FILE)
data.head()

,Unnamed: 0,sepal length,sepal width,petal length,petal width,class
0,0,5.1,3.5,1.4,0.2,0.0
1,1,4.9,3.0,1.4,0.2,0.0
2,2,4.7,3.2,1.3,0.2,0.0
3,3,4.6,3.1,1.5,0.2,0.0
4,4,5.0,3.6,1.4,0.2,0.0


In [4]:
# split data into test and training
train_data = data.sample(frac=0.8, random_state=RANDOM_STATE)
test_data = data.drop(train_data.index)

test_data = test_data.drop(['class'],axis=1)
test_data.head()

,Unnamed: 0,sepal length,sepal width,petal length,petal width
5,5,5.4,3.9,1.7,0.4
12,12,4.8,3.0,1.4,0.1
17,17,5.1,3.5,1.4,0.3
19,19,5.1,3.8,1.5,0.3
20,20,5.4,3.4,1.7,0.2


In [5]:
# put train data into csv
TRAIN_DATA_FILE = './aws_sagemaker/scikit-learn/classification/iris_parameterized/data/train_data.csv'
train_data.to_csv(TRAIN_DATA_FILE, index=False, header=True)

# put test data into csv
TEST_DATA_FILE = './aws_sagemaker/scikit-learn/classification/iris_parameterized/data/test_data.csv'
test_data.to_csv(TEST_DATA_FILE, index=False, header=True)

### Push data to S3 bucket

#### Define S3 Paths

In [6]:
BUCKET_NAME = 'snowflake-getting-started'
BASE_PREFIX = 'iris'

INPUT_DATA_PREFIX = BASE_PREFIX+'/data/input'
TRAIN_DATA_PREFIX = BASE_PREFIX+'/data/input/train'
TEST_DATA_PREFIX = BASE_PREFIX+'/data/input/test'

EXPERIMENTS_OUTPUT_LOC = 's3://'+BUCKET_NAME+'/'+BASE_PREFIX+'/experiments'
EXPERIMENTS_TRAINING_METRICS_PREFIX = BASE_PREFIX + '/experiments'

print ('Experiment metadata would be published at -',EXPERIMENTS_OUTPUT_LOC)

Experiment metadata would be published at - s3://snowflake-getting-started/iris/experiments


#### Upload data to S3

In [7]:
print ('Uploading train data to s3')

s3_input_data_path = session.upload_data(path=DATA_FILE, 
                           bucket=BUCKET_NAME, 
                           key_prefix=INPUT_DATA_PREFIX)

print ('Input data uploaded to -', s3_input_data_path)

Uploading train data to s3
Input data uploaded to - s3://snowflake-getting-started/iris/data/input/iris.csv


### Step 1 - Setup an Experiment

### Create an Experiment

In [8]:
from smexperiments.experiment import Experiment
import time

EXPERIMENT_NAME=f"experiment-iris-classification-model-{int(time.time())}"

iris_experiment = Experiment.create(
    experiment_name=EXPERIMENT_NAME,
    description="Classification of iris flowers", 
    sagemaker_boto_client=sm)

EXP_DEBUGGING_OUTPUTS=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/debugging'
EXP_TRAINED_MODELS=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/trained_models'
EXP_SOURCE_CODE=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/code'
EXPERIMENTS_TRAINING_METRICS_PREFIX = EXPERIMENTS_TRAINING_METRICS_PREFIX + '/'+EXPERIMENT_NAME+'/trained_models'

print ('Experiment debugging data available at -',EXP_DEBUGGING_OUTPUTS)
print ('Experiment trained moddels available at -',EXP_TRAINED_MODELS)
print ('Experiment source code available at -', EXP_SOURCE_CODE)
print ('Experiment training metrics avaiable at -',EXPERIMENTS_TRAINING_METRICS_PREFIX)

Experiment debugging data available at - s3://snowflake-getting-started/iris/experiments/experiment-iris-classification-model-1595880407/debugging
Experiment trained moddels available at - s3://snowflake-getting-started/iris/experiments/experiment-iris-classification-model-1595880407/trained_models
Experiment source code available at - s3://snowflake-getting-started/iris/experiments/experiment-iris-classification-model-1595880407/code
Experiment training metrics avaiable at - iris/experiments/experiment-iris-classification-model-1595880407/trained_models


### Data Pre-Processing

#### Pre-processing tasks

    This is where we would do data pre-processing and convert intput data into some features. These features would then be uploaded to S3 and would be
    used for model training.

#### Create Tracker for Pre-Processing

    Here we are doing two things:
    1. We are creating a trial-component for 'preprocessing'
    2. We are creating a tracker to how it tracks the trial-component. This includes:-
        2.1 the artefacts generated for this trial component - for e.g. the S3 location where the artefacts would be stored
        2.2 Input data location (S3)
        2.3 Output data / Features Location (S3)
        2.4 Significant parameters from pre-processing.

In [9]:
from smexperiments.tracker import Tracker

preprocessing_trial_component_name = 'Data-Preprocessing'
preprocessing_trial_metadata_prefix = BASE_PREFIX+'/'+EXPERIMENT_NAME+'/metadata'+'/'+preprocessing_trial_component_name        

with Tracker.create(display_name=preprocessing_trial_component_name, 
                    sagemaker_boto_client=sm,
                    artifact_bucket=BUCKET_NAME,
                    artifact_prefix=preprocessing_trial_metadata_prefix) as tracker:
    tracker.log_parameters({
         #Record a single parameter value for this trial component.
        "pre_processing_param": 'pre_processing_param_value'
    })
    # Record a single input artifact for this trial component.
    # we can log the s3 uri to the dataset we just uploaded
    tracker.log_input(name="iris-dataset", media_type="s3/uri", value=s3_input_data_path)
    tracker.log_output(name="iris-dataset", media_type="s3/uri", value=s3_input_data_path)
    
# we are taking the trial component which was created earlir for preprocessing. since for different training trails
# our pre-processing trial is the same hence we just attach that preprocessing component to this trial.
preprocessing_trial_component = tracker.trial_component
    
print ('Preprocessing trial metadata would be stored at -',preprocessing_trial_metadata_prefix)    

Preprocessing trial metadata would be stored at - iris/experiment-iris-classification-model-1595880407/metadata/Data-Preprocessing


### Step 2 - Track Experiment

    We next iterate through a loop to run different trials with the hyper parameters. For each iteration we do the following:
    1. Create a Trial, since each iteration represents a trial we are doing.
    2. Add the pre-processing trial component to this trial
    3. Specify a Trial Component for the training job.

In [10]:
from sagemaker.sklearn import SKLearn
from smexperiments.trial import Trial
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig, CollectionConfig
import csv

for i, num_max_iter in enumerate([5,10]):
    
    # create trial
    trial_name = f"trial-iris-classification-model-{num_max_iter}-max-iter-{int(time.time())}"
    trial_metadata_prefix = BASE_PREFIX+'/'+EXPERIMENT_NAME+'/metadata'+trial_name
    print ('Trial metadata would be stored at -',trial_metadata_prefix)
    
    iris_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=iris_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    
    # associate the proprocessing trial component with the current trial
    iris_trial.add_trial_component(preprocessing_trial_component)
    
    executor = SKLearn(entry_point='./aws_sagemaker/scikit-learn/classification/iris_parameterized/train.py',
                      train_instance_type='ml.c4.xlarge',
                      sagemaker_session = session,
                      role = role,
                      code_location  = EXP_SOURCE_CODE,
                      hyperparameters = {
                          'max_iter':num_max_iter,
                          'class_weight':'balanced'
                      },
                      input_mode='File',
                      metric_definitions=[
                        {'Name':'test:f1-score', 'Regex':'Test F1-Score: (.*)'},
                        {'Name':'test:accuracy', 'Regex':'Test Accuracy: (.*)'}
                      ],
                      enable_sagemaker_metrics=True,
                      output_path = EXP_TRAINED_MODELS
                )
    
    trial_component_training_job = "training-job-iris-classification-model-{}".format(int(time.time()))
    executor.fit(
        inputs={'training': s3_input_data_path},
        job_name=trial_component_training_job,
        experiment_config={
            "TrialName": iris_trial.trial_name,
            "TrialComponentDisplayName": "Model-Training",
        },
        logs=True,
        wait=True
    )
        
    ########################################### Persisting Training Metrics ##########################################
    # dump training metrics into a csv file
    training_metrics = executor.jobs[-1].describe()
    training_metrics_file_name = './aws_sagemaker/scikit-learn/classification/iris_parameterized/'+trial_component_training_job+'.csv'
    
    with open(training_metrics_file_name, 'w') as f:
        for key in training_metrics.keys():
            f.write("%s,%s\n"%(key,training_metrics[key]))
        
    # upload training metrics csv file to S3 
    s3_metrics_data_path = session.upload_data(path=training_metrics_file_name, 
                                               bucket=BUCKET_NAME, 
                                               key_prefix=EXPERIMENTS_TRAINING_METRICS_PREFIX+'/'+trial_component_training_job)

    print ('metrics data uploaded to -', s3_metrics_data_path)
   
    ########################################### Persisting Transformed Data ##########################################
    
    # give it a while before dispatching the next training job
    time.sleep(2)

Trial metadata would be stored at - iris/experiment-iris-classification-model-1595880407/metadatatrial-iris-classification-model-5-max-iter-1595880407


INFO:sagemaker:Creating training-job with name: training-job-iris-classification-model-1595880407


2020-07-27 20:06:48 Starting - Starting the training job...
2020-07-27 20:06:50 Starting - Launching requested ML instances......
2020-07-27 20:08:00 Starting - Preparing the instances for training......
2020-07-27 20:09:21 Downloading - Downloading input data
2020-07-27 20:09:21 Training - Downloading the training image...
2020-07-27 20:09:47 Uploading - Uploading generated training model2020-07-27 20:09:42,472 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-07-27 20:09:42,474 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-27 20:09:42,485 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-07-27 20:09:42,772 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-07-27 20:09:42,772 sagemaker-containers INFO     Generating setup.cfg
2020-07-27 20:09:42,772 sagemaker-containers INFO     Generating MANIFEST.in
2020-07-27 20:09:42,772 sag

INFO:sagemaker:Creating training-job with name: training-job-iris-classification-model-1595880639


Trial metadata would be stored at - iris/experiment-iris-classification-model-1595880407/metadatatrial-iris-classification-model-10-max-iter-1595880639
2020-07-27 20:10:39 Starting - Starting the training job...
2020-07-27 20:10:41 Starting - Launching requested ML instances......
2020-07-27 20:11:47 Starting - Preparing the instances for training......
2020-07-27 20:13:04 Downloading - Downloading input data
2020-07-27 20:13:04 Training - Downloading the training image...
2020-07-27 20:13:37 Uploading - Uploading generated training model
2020-07-27 20:13:37 Completed - Training job completed
2020-07-27 20:13:24,985 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-07-27 20:13:24,987 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-27 20:13:24,998 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-07-27 20:13:25,273 sagemaker-containers INFO     Module train does not provide a se

### Compare the model training runs for an experiment

Now we will use the analytics capabilities of Python SDK to query and compare the training runs for identifying the best model produced by our experiment. You can retrieve trial components by using a search expression.

### Some Simple Analyses

In [17]:
search_expression = {
    "Filters":[
        {
            "Name": "TrialComponentName",
            "Operator": "Contains",
            "Value": "iris",
        }
    ],
}
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.session import Session

trial_component_analytics = ExperimentAnalytics(
    sagemaker_session=Session(boto3.Session(), sm), 
    experiment_name=iris_experiment.experiment_name,
    search_expression=search_expression,
    sort_by="metrics.test:f1-score.max",
    metric_names=['test:f1-score'],
    sort_order="Descending",    
    #metric_names=['test:f1-score'],
    parameter_names=['max_iter', 'class_weight']
)
trial_component_analytics.dataframe()

,TrialComponentName,DisplayName,SourceArn,class_weight,max_iter,test:f1-score - Min,test:f1-score - Max,test:f1-score - Avg,test:f1-score - StdDev,test:f1-score - Last,test:f1-score - Count
0,training-job-iris-classification-model-1595880...,Model-Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,"""balanced""",10.0,1.000000,1.000000,1.000000,0.0,1.000000,1
1,training-job-iris-classification-model-1595880...,Model-Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,"""balanced""",5.0,0.868421,0.868421,0.868421,0.0,0.868421,1


In [18]:
lineage_table = ExperimentAnalytics(
    sagemaker_session=Session(boto3.Session(), sm), 
    search_expression={
        "Filters":[{
            "Name": "Parents.TrialName",
            "Operator": "Contains",
            "Value": 'iris-training-job'
        },
        {
            "Name": "DisplayName",
            "Operator": "Equals",
            "Value": "Training",
        }]
    },
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_table.dataframe()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,class_weight,max_iter,sagemaker_container_log_level,...,test:f1-score - Avg,test:f1-score - StdDev,test:f1-score - Last,test:f1-score - Count,test:accuracy - Min,test:accuracy - Max,test:accuracy - Avg,test:accuracy - StdDev,test:accuracy - Last,test:accuracy - Count
0,iris-training-job-1595704800-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",10.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,iris-training-job-1595705024-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",20.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,iris-training-job-1595705505-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",10.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,iris-training-job-1595705698-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",20.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,iris-training-job-1595705969-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",20.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,iris-training-job-1595707279-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",10.0,20.0,...,0.921053,0.0,0.921053,1.0,0.921053,0.921053,0.921053,0.0,0.921053,1.0
6,iris-training-job-1595707533-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",20.0,20.0,...,0.947368,0.0,0.947368,1.0,0.947368,0.947368,0.947368,0.0,0.947368,1.0
7,iris-training-job-1595711070-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",10.0,20.0,...,0.868421,0.0,0.868421,1.0,0.868421,0.868421,0.868421,0.0,0.868421,1.0
8,iris-training-job-1595711296-aws-training-job,Training,arn:aws:sagemaker:eu-west-1:951135073253:train...,141502667606.dkr.ecr.eu-west-1.amazonaws.com/s...,1.0,ml.c4.xlarge,30.0,"""balanced""",20.0,20.0,...,0.947368,0.0,0.947368,1.0,0.947368,0.947368,0.947368,0.0,0.947368,1.0


### Step - Real Time Predictions

#### Deploy Model to an Endpoint

In [14]:
#predictor = executor.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
#print('\nModel Deployed!')
#print (predictor.endpoint)

#### Realtime Inference

    Lookup the predictor via the endpoint name & call predict on it

In [15]:
#import pandas as pd
#from sklearn.model_selection import train_test_split

#data = pd.read_csv(DATA_FILE,engine='python')
#X = data.iloc[:,1:5]
#y = data.iloc[:,5]

#train_x, test_x, train_y, test_y = train_test_split(X,y)
#pred_y = predictor.predict(test_x)
#print (pred_y)
#print (test_y)